In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Arrhythmia/GSE97455'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Plasma Circular RNAs, Hsa_circRNA_025016, Predicts Postoperative Atrial Fibrillation After Isolated Off-Pump Coronary Artery Bypass Grafting"
!Series_summary	"Rationale: Circular RNAs are pervasively expressed in highly diverged eukaryotes. Circular RNAs are more stable in body fluids, however, the link between circular RNA and onset of atrial fibrillation has never been investigated."
!Series_summary	""
!Series_summary	"Objective: To identify plasma circular RNAs for diagnosing onset of atrial fibrillation after the cardiac surgery."
!Series_summary	""
!Series_summary	"Methods and Results: Plasma circular RNAs expression was investigated in participants underwent isolated off-pump coronary artery bypass grafting. First, we used microarray to screen 15 circular RNAs in 30 plasma samples for diagnosing new onset of atrial fibrillation. Quantitative polymerase chain reaction assay was then applied to evaluate the expression of selected circular RNAs

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check if the dataset contains gene expression data
is_gene_available = True  # Since circular RNA expression is a type of gene expression

# Data Availability
sample_characteristics = {
    0: ['subject status: patients without postoperative atrial fibrilltion (no-PoAF)', 'subject status: patients with postoperative atrial fibrilltion (PoAF)'],
    1: ['gender: male', 'gender: female'],
    2: ['tissue: plasma'],
    3: ['molecule subtype: circular RNA']
}

# Checking availability for 'Arrhythmia'
if len(sample_characteristics[0]) > 1:
    trait_row = 0  # 'subject status' key

# Checking availability for 'age'
# Age data is not explicitly mentioned in the characteristics provided
age_row = None

# Checking availability for 'gender'
if len(sample_characteristics[1]) > 1:
    gender_row = 1  # 'gender' key

# Data Type Conversion Functions
def convert_trait(value):
    header, val = value.split(': ')
    if val == 'patients without postoperative atrial fibrilltion (no-PoAF)':
        return 0
    elif val == 'patients with postoperative atrial fibrilltion (PoAF)':
        return 1
    return None  # Unknown or unexpected value

def convert_age(value):
    # Age data is not available
    return None

def convert_gender(value):
    header, val = value.split(': ')
    if val == 'female':
        return 0
    elif val == 'male':
        return 1
    return None  # Unknown or unexpected value

# Save cohort information
save_cohort_info('GSE97455', './preprocessed/Arrhythmia/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Arrhythmia', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Arrhythmia/trait_data/GSE97455.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM2564863': [0, 1], 'GSM2564864': [0, 1], 'GSM2564865': [0, 1], 'GSM2564866': [0, 1], 'GSM2564867': [0, 1], 'GSM2564868': [0, 0], 'GSM2564869': [0, 1], 'GSM2564870': [0, 0], 'GSM2564871': [0, 1], 'GSM2564872': [0, 0], 'GSM2564873': [0, 1], 'GSM2564874': [0, 1], 'GSM2564875': [0, 1], 'GSM2564876': [0, 0], 'GSM2564877': [0, 1], 'GSM2564878': [1, 1], 'GSM2564879': [1, 0], 'GSM2564880': [1, 1], 'GSM2564881': [1, 0], 'GSM2564882': [1, 1], 'GSM2564883': [1, 1], 'GSM2564884': [1, 1], 'GSM2564885': [1, 1], 'GSM2564886': [1, 1], 'GSM2564887': [1, 1], 'GSM2564888': [1, 1], 'GSM2564889': [1, 1], 'GSM2564890': [1, 0], 'GSM2564891': [1, 1], 'GSM2564892': [1, 0]}


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['ASCRP3000001', 'ASCRP3000002', 'ASCRP3000003', 'ASCRP3000004',
       'ASCRP3000005', 'ASCRP3000006', 'ASCRP3000007', 'ASCRP3000008',
       'ASCRP3000009', 'ASCRP3000010', 'ASCRP3000011', 'ASCRP3000012',
       'ASCRP3000013', 'ASCRP3000014', 'ASCRP3000015', 'ASCRP3000016',
       'ASCRP3000017', 'ASCRP3000018', 'ASCRP3000019', 'ASCRP3000020'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ASCRP3000001', 'ASCRP3000002', 'ASCRP3000003', 'ASCRP3000004', 'ASCRP3000005'], 'circRNA': ['hsa_circRNA_082501', 'hsa_circRNA_407247', 'hsa_circRNA_007418', 'hsa_circRNA_407121', 'hsa_circRNA_407087'], 'TYPE': ['circRNA', 'circRNA', 'circRNA', 'circRNA', 'circRNA'], 'BUILD': ['HG19', 'HG19', 'HG19', 'HG19', 'HG19'], 'SEQUENCE': ['AAAAAAACAACCAAAAAATGTTCAACAGCATGAGAAGGTTCAGAAAGCCAGTACAGAGGG', 'AAAAAACGAAGAAAAAGAGACACCCAGCTCACCTCCAAGTTTGCCTGCAGGAGCCGGCTC', 'AAAAAACTAGAAATGTGTTCAGAAATTAAAGGTCCACAGAAGGAGGGCCTGTCCTCCCAA', 'AAAAAAGAAGTCCTCGGATACGCTGAAACTGCAGAAGAAAGCAAAAAAAGGTGGGACCAG', 'AAAAAAGGAACAGGAAAACTACATCTCAGACCCAACTATCTTATGGAGGGCTTATCCCCA']}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. When analyzing a gene expression dataset, identify the keys
identifier_key = 'ID'
gene_symbol_key = 'circRNA'

# 2. Get the dataframe storing the mapping between probe IDs and genes
gene_mapping = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping with the 'apply_gene_mapping' function from the library
gene_data = apply_gene_mapping(gene_data, gene_mapping)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Arrhythmia/gene_data/GSE97455.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Arrhythmia')

# Define trait_biased
trait_biased = trait_biased

# 4. Save the cohort information.
save_cohort_info('GSE97455', './preprocessed/Arrhythmia/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Arrhythmia/GSE97455.csv'
    unbiased_merged_data.to_csv(csv_path)


No gene data in the dataframe
